In [1]:
from itertools import product

# 장갑
+ special case:
+ 쓸윈부 Y Y
+ X 쓸윈부 Y
+ 무시 무시 Y

In [2]:
first = {
    'STR : +12%': 0.1,
    'DEX : +12%': 0.1,
    'INT : +12%': 0.1,
    'LUK : +12%': 0.1,
    '최대 HP : +12%': 0.1,
    '최대 MP : +12%': 0.1,
    '크리티컬 데미지 : +8%': 0.1,
    '올스탯 : +9%': 0.075,
    '피격 시 10% 확률로 데미지의 20% 무시': 0.075,
    '피격 시 10% 확률로 데미지의 40% 무시': 0.075,
    '<쓸만한 윈드 부스터> 스킬 사용 가능': 0.075,
}

second = {
    'STR : +9%': 1/14,
    'DEX : +9%': 1/14,
    'INT : +9%': 1/14,
    'LUK : +9%': 1/14,
    '최대 HP : +9%': 3/35,
    '최대 MP : +9%': 3/35,
    '올스탯 : +6%': 2/35,
    'STR : +32': 1/70,
    'DEX : +32': 1/70,
    'INT : +32': 1/70,
    'LUK : +32': 1/70,
    '피격 시 5% 확률로 데미지의 20% 무시': 2/35,
    '피격 시 5% 확률로 데미지의 40% 무시': 2/35,
    'HP 회복 아이템 및 회복 스킬 효율 : +30%': 2/35,
    '<쓸만한 샤프 아이즈> 스킬 사용 가능': 2/35,
    'STR : +12%': 0.02,
    'DEX : +12%': 0.02,
    'INT : +12%': 0.02,
    'LUK : +12%': 0.02,
    '최대 HP : +12%': 0.02,
    '최대 MP : +12%': 0.02,
    '크리티컬 데미지 : +8%': 0.02,
    '올스탯 : +9%': 0.015,
    '피격 시 10% 확률로 데미지의 20% 무시': 0.015,
    '피격 시 10% 확률로 데미지의 40% 무시': 0.015,
    '<쓸만한 윈드 부스터> 스킬 사용 가능': 0.015,
}

third = {
    'STR : +9%': 0.95/56 * 5,
    'DEX : +9%': 0.95/56 * 5,
    'INT : +9%': 0.95/56 * 5,
    'LUK : +9%': 0.95/56 * 5,
    '최대 HP : +9%': 0.95/56 * 6,
    '최대 MP : +9%': 0.95/56 * 6,
    '올스탯 : +6%': 0.95/56 * 4,
    'STR : +32': 0.95/56,
    'DEX : +32': 0.95/56,
    'INT : +32': 0.95/56,
    'LUK : +32': 0.95/56,
    '피격 시 5% 확률로 데미지의 20% 무시': 0.95/56 * 4,
    '피격 시 5% 확률로 데미지의 40% 무시': 0.95/56 * 4,
    'HP 회복 아이템 및 회복 스킬 효율 : +30%': 0.95/56 * 4,
    '<쓸만한 샤프 아이즈> 스킬 사용 가능': 0.95/56 * 4,
    'STR : +12%': 0.005,
    'DEX : +12%': 0.005,
    'INT : +12%': 0.005,
    'LUK : +12%': 0.005,
    '최대 HP : +12%': 0.005,
    '최대 MP : +12%': 0.005,
    '크리티컬 데미지 : +8%': 0.005,
    '올스탯 : +9%': 0.00375,
    '피격 시 10% 확률로 데미지의 20% 무시': 0.00375,
    '피격 시 10% 확률로 데미지의 40% 무시': 0.00375,
    '<쓸만한 윈드 부스터> 스킬 사용 가능': 0.00375,
}

In [3]:
def glove_pmf_list():
    second_without_useful = 1 - sum(second[option] for option in second.keys() if '쓸만한' in option)
    third_without_useful = 1 - sum(third[option] for option in third.keys() if '쓸만한' in option)
    third_without_ignore = 1 - sum(third[option] for option in third.keys() if '무시' in option)

    options_list = product(first.keys(), second.keys(), third.keys())
    result = []; result_append = result.append  # 6,800

    for options in options_list:
        useful_count = sum('쓸만한' in option for option in options)
        ignore_count = sum('무시' in option for option in options)
        if useful_count >= 2 or ignore_count == 3:
            continue
        op1, op2, op3 = options

        prob1 = first[op1]
        prob2 = second[op2]
        prob3 = third[op3]

        if '쓸만한' in op1:
            prob2 /= second_without_useful
            prob3 /= third_without_useful
        elif '쓸만한' in op2:
            prob3 /= third_without_useful
        
        if ignore_count==2 and '무시' not in op3:  # first, second option is '무시'-like
            prob3 /= third_without_ignore

        prob = prob1*prob2*prob3

        result_append((*options, prob))
    
    return result

In [4]:
result = glove_pmf_list()
total_prob = 0

for (op1, op2, op3, prob) in result:
    if sum('크리티컬 데미지' in option for option in (op1, op2, op3)) >= 2:
        total_prob += prob

print(f'확률: {total_prob:6.3%}, 기대값: {23400000/total_prob:14,.0f}')

확률: 0.258%, 기대값:  9,055,766,832


## conditional probability

In [5]:
critical_list = [temp for temp in result if sum('크리티컬 데미지' in option for option in temp[:-1]) >= 1]

critical_1_prob = sum(prob for (_, _, _, prob) in critical_list)
critical_2_prob = 0
for (op1, op2, op3, prob) in critical_list:
    if ('크리티컬 데미지' in op2 or '크리티컬 데미지' in op3) and ('크리티컬 데미지' in op1):
        critical_2_prob += prob

In [6]:
critical_2_prob / critical_1_prob

0.020333409442219175